In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys, getopt
import csv
import pandas as pd
from collections import Counter
import re
import time
import timeit

In [2]:
import os
path = os.getcwd()
print(path)
os.chdir(r"C:\Users\stanma02\Desktop\Final Project\WebScraper and Data")
path = os.getcwd()
print(path)

C:\Users\stanma02\Desktop\Final Project\WebScraper and Data
C:\Users\stanma02\Desktop\Final Project\WebScraper and Data


In [3]:
def Patientwebscraper(url):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    res = requests.get(url, headers=headers)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')

    all_tables = soup.findAll("tbody")
    results=soup.find(id="team_stats").get_text()
    results2=soup.find(id="team_stats_extra").get_text()

    x=re.split("\n|\t|\xa0—\xa0|Cards|\xa0",results)
    test_list = list(filter(None, x))
    x2=re.split("\n|\xa0|\t|—|\xa0—\xa0",results2)
    test_list2 = list(filter(None, x2))

    results=soup.find(id="team_stats")
    results2=soup.find(id="team_stats_extra")
    
    Cards= results.find_all(class_='cards')
    Homecards= list(filter (lambda x: x not in ["yellow_card","red_card"],Cards[0]))
    Awaycards= list(filter (lambda x: x not in ["yellow_card","red_card"],Cards[1]))
    d= pd.DataFrame(data=Counter(Homecards).values()).T.rename(columns={0:"YellowCards",1:"RedCards"})
    e= pd.DataFrame(data=Counter(Awaycards).values()).T.rename(columns={0:"YellowCards",1:"RedCards"})

    if len(d.columns) < 2 :
        if len(d.columns)==0:
            d= {'YellowCards':[0], 'RedCards': [0]}
            d= pd.DataFrame(d)
        if len(d.columns)==1 and d.columns == "YellowCards":
            d["RedCards"] = 0
        if len(d.columns)==1 and d.columns == "RedCards":
            d["YellowCards"] = 0
        else:
            d

    if len(e.columns) < 2 :
        if len(e.columns)==0:
            e= {'YellowCards':[0], 'RedCards': [0]}
            e= pd.DataFrame(e)
        if len(e.columns)==1 and e.columns == "YellowCards":
            e["RedCards"] = 0
        if len(e.columns)==1 and e.columns == "RedCards":
            e["YellowCards"] = 0
        else:
            e

    split_string = test_list[7].split(" of ", 1)
    split_string1 = test_list[12].split(" of ", 1)
    split_string3= test_list2[2].split("Fouls")
    split_string4= test_list2[8].split("Tackles")
    split_string5= test_list2[9].split("Interceptions")
    split_string6 = test_list[10].split(" of ", 1)
    split_string7 = test_list[15].split(" of ", 1)

    data3= [test_list[5],split_string6[0],split_string6[1], split_string7[0],split_string7[1],
      split_string3[1],split_string4[1],split_string5[1]]
    data4= [test_list[4],split_string[0],split_string[1], split_string1[0],split_string1[1],
      split_string3[0],split_string4[0],split_string5[0]]
    HomeDF=pd.DataFrame(data4).T.rename(columns= {0:'HomePossession', 1:'HomePassesCompleted', 2:'HomePassesAttempts', 3:'HomeShotsonTarget',
                                             4:'HomeShots', 5:'HomeFouls', 6:'HomeTackles', 
                                             7:'HomeInterceptions'})
    d.rename(columns= {'YellowCards': 'HomeYellow','RedCards':'HomeRed'}, inplace=True)
    HomeFinal=pd.concat([HomeDF,d], axis=1)
    AwayDF=pd.DataFrame(data3).T.rename(columns= {0:'AwayPossession', 1:'AwayPassesCompleted', 2:'AwayPassesAttempts', 3:'AwayShotsonTarget',
                                              4:'AwayShots', 5:'AwayFouls', 6:'AwayTackles', 
                                              7:'AwayInterceptions'})
    e.rename(columns= {'YellowCards': 'AwayYellow','RedCards':'AwayRed'}, inplace=True)
    AwayFinal=pd.concat([AwayDF,e], axis=1)
    MatchStats=pd.concat([HomeFinal,AwayFinal],axis=1)
    time.sleep(3.01)
    return MatchStats

In [53]:
df = pd.read_csv("BasicTest.csv",encoding='cp1252')
df.head()

,Match_ID,League,Season,Wk,Day,Date,Time,Home,Score,Away,...,Notes,Venue Key,VAR Key,Stadium_Capacity,VAR,HomeGoal,AwayGoal,HomeResult,StadiumUtilization,UtilizationBuckets
0,9036,SerieA,20-21,29,Sat,2021-04-03,18:00 (12:00),Torino,2-2,Juventus,...,NaN,Torino Stadio Olimpico di Torino,SerieA 20-21,27958,1,2,2,Tie,0.0,No Fans
1,9037,SerieA,20-21,29,Sat,2021-04-03,20:45 (14:45),Bologna,0-1,Inter,...,NaN,Bologna Stadio Renato Dall'Ara,SerieA 20-21,36462,1,0,1,Loss,0.0,No Fans
2,9038,SerieA,20-21,3,Wed,2021-04-07,18:45 (12:45),Juventus,2-1,Napoli,...,NaN,Juventus Allianz Stadium,SerieA 20-21,41507,1,2,1,Win,0.0,No Fans
3,9039,SerieA,20-21,28,Wed,2021-04-07,18:45 (12:45),Inter,2-1,Sassuolo,...,NaN,Inter Stadio Giuseppe Meazza,SerieA 20-21,75923,1,2,1,Win,0.0,No Fans
4,9040,SerieA,20-21,30,Sat,2021-04-10,15:00 (09:00),Spezia,3-2,Crotone,...,NaN,Spezia Stadio Dino Manuzzi,SerieA 20-21,10336,1,3,2,Win,0.0,No Fans


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Match_ID            95 non-null     int64  
 1   League              95 non-null     object 
 2   Season              95 non-null     object 
 3   Wk                  95 non-null     int64  
 4   Day                 95 non-null     object 
 5   Date                95 non-null     object 
 6   Time                95 non-null     object 
 7   Home                95 non-null     object 
 8   Score               95 non-null     object 
 9   Away                95 non-null     object 
 10  Attendance          95 non-null     float64
 11  Venue               95 non-null     object 
 12  Referee             95 non-null     object 
 13  Match Report        95 non-null     object 
 14  Notes               0 non-null      float64
 15  Venue Key           95 non-null     object 
 16  VAR Key   

In [55]:
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= Patientwebscraper(row["Match Report"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)  

In [56]:
TotalMatchData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   HomePossession       95 non-null     object
 1   HomePassesCompleted  95 non-null     object
 2   HomePassesAttempts   95 non-null     object
 3   HomeShotsonTarget    95 non-null     object
 4   HomeShots            95 non-null     object
 5   HomeFouls            95 non-null     object
 6   HomeTackles          95 non-null     object
 7   HomeInterceptions    95 non-null     object
 8   HomeYellow           95 non-null     int64 
 9   HomeRed              95 non-null     int64 
 10  AwayPossession       95 non-null     object
 11  AwayPassesCompleted  95 non-null     object
 12  AwayPassesAttempts   95 non-null     object
 13  AwayShotsonTarget    95 non-null     object
 14  AwayShots            95 non-null     object
 15  AwayFouls            95 non-null     object
 16  AwayTackle

In [13]:
df=df.iloc[:26]

In [57]:
start9036to9130=pd.concat([df,TotalMatchData],axis=1)

In [58]:
start9036to9130.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Match_ID             95 non-null     int64  
 1   League               95 non-null     object 
 2   Season               95 non-null     object 
 3   Wk                   95 non-null     int64  
 4   Day                  95 non-null     object 
 5   Date                 95 non-null     object 
 6   Time                 95 non-null     object 
 7   Home                 95 non-null     object 
 8   Score                95 non-null     object 
 9   Away                 95 non-null     object 
 10  Attendance           95 non-null     float64
 11  Venue                95 non-null     object 
 12  Referee              95 non-null     object 
 13  Match Report         95 non-null     object 
 14  Notes                0 non-null      float64
 15  Venue Key            95 non-null     objec

In [47]:
#Might need this might not
start9036to9130=start9036to9130.iloc[:, :-1]

In [59]:
start9036to9130.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Match_ID             95 non-null     int64  
 1   League               95 non-null     object 
 2   Season               95 non-null     object 
 3   Wk                   95 non-null     int64  
 4   Day                  95 non-null     object 
 5   Date                 95 non-null     object 
 6   Time                 95 non-null     object 
 7   Home                 95 non-null     object 
 8   Score                95 non-null     object 
 9   Away                 95 non-null     object 
 10  Attendance           95 non-null     float64
 11  Venue                95 non-null     object 
 12  Referee              95 non-null     object 
 13  Match Report         95 non-null     object 
 14  Notes                0 non-null      float64
 15  Venue Key            95 non-null     objec

In [60]:
start9036to9130=start9036to9130.set_index('Match_ID')

In [61]:
print(start9036to9130.shape)

(95, 43)


In [62]:
start9036to9130.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\start9036to9130.csv')

# Create Dataframes for Each Subset and Combine Each Season

## Bundesliga 16-17

In [54]:
print(Upto103.shape)
print(start104to206.shape)
print(start207to306.shape)

(103, 43)
(103, 43)
(100, 43)


In [55]:
frames = [Upto103, start104to206, start207to306]
FinalData=pd.concat(frames)

In [57]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Bundesliga1617.csv')

## Bundesliga 17-18

In [35]:
print(start307to409.shape)
print(start410to522.shape)
print(start523to612.shape)
#print(.shape)

(103, 43)
(113, 43)
(90, 43)


In [36]:
frames = [start307to409, start410to522,start523to612]
FinalData=pd.concat(frames)

In [37]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Bundesliga1718.csv')

## Bundesliga 18-19

In [80]:
print(start613to714.shape)
print(start715to816.shape)
print(start817to918.shape)

(102, 43)
(102, 43)
(102, 43)


In [81]:
frames = [start613to714, start715to816,start817to918]
FinalData=pd.concat(frames)

In [82]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Bundesliga1819.csv')

## Bundesliga 19-20

In [124]:
print(start919to1020.shape)
print(start1021to1122.shape)
print(start1123to1224.shape)

(102, 43)
(102, 43)
(102, 43)


In [125]:
frames = [start919to1020, start1021to1122, start1123to1224]
FinalData=pd.concat(frames)

In [126]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Bundesliga1920.csv')

## Bundesliga 20-21

In [158]:
print(start1225to1326.shape)
print(start1327to1428.shape)
print(start1429to1530.shape)

(102, 43)
(102, 43)
(102, 43)


In [159]:
frames = [start1225to1326, start1327to1428,start1429to1530]
FinalData=pd.concat(frames)

In [160]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Bundesliga2021.csv')

## EPL 16-17

In [216]:
print(start1531to1657.shape)
print(start1658to1759.shape)
print(start1760to1861.shape)
print(start1862to1910.shape)

(127, 43)
(102, 43)
(102, 43)
(49, 43)


In [218]:
frames = [start1531to1657, start1658to1759,start1760to1861,start1862to1910]
FinalData=pd.concat(frames)

In [219]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\EPL1617.csv')

## EPL 17-18

In [52]:
print(start1911to2006.shape)
print(start2007to2102.shape)
print(start2103to2197.shape)
print(start2198to2290.shape)

NameError: name 'start1911to2006' is not defined

In [48]:
frames = [start1911to2006, start2007to2102, start2103to2197, start2198to2290]
FinalData=pd.concat(frames)

In [49]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\EPL1718.csv')

## EPL 18-19

In [53]:
print(start2291to2385.shape)
print(start2385to2480.shape)
print(start2481to2575.shape)
print(start2576to2670.shape)

(94, 43)
(96, 43)
(95, 43)
(95, 43)


In [54]:
frames = [start2291to2385, start2385to2480,start2481to2575,start2576to2670]
FinalData=pd.concat(frames)

In [55]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\EPL1819.csv')

## EPL 19-20

In [57]:
start2671to2764=start2671to2764.set_index('Match_ID')

In [58]:
print(start2671to2764.shape)
print(start2765to2859.shape)
print(start2860to2954.shape)
print(start2955to3050.shape)

(94, 43)
(95, 43)
(95, 43)
(96, 43)


In [59]:
frames = [start2671to2764, start2765to2859,start2860to2954, start2955to3050]
FinalData=pd.concat(frames)

In [60]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\EPL1920.csv')

## EPL 20-21

In [49]:
print(start3051to3146.shape)
print(start3147to3241.shape)
print(start3242to3336.shape)
print(start3337to3430.shape)

(96, 43)
(95, 43)
(95, 43)
(94, 43)


In [50]:
frames = [start3051to3146, start3147to3241, start3242to3336, start3337to3430]
FinalData=pd.concat(frames)

In [51]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\EPL2021.csv')

## LaLiga 16-17

In [87]:
start3526to3562 = pd.read_csv("start3526to3562.csv",encoding='cp1252')
start3526to3562.head()
start3526to3562=start3526to3562.set_index('Match_ID')

In [128]:
print(start3431to3525.shape)
print(start3526to3562.shape)
print(start3563to3620.shape)
print(start3621to3715.shape)
print(start3716to3810.shape)

(95, 43)
(37, 43)
(58, 43)
(95, 43)
(95, 43)


In [129]:
frames = [start3431to3525, start3526to3562, start3563to3620, start3621to3715, start3716to3810]
FinalData=pd.concat(frames)

In [130]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\LaLiga1617.csv')

## LaLiga 17-18

In [51]:
print(start3811to3905.shape)
print(start3906to4000.shape)
print(start4001to4095.shape)
print(start4096to4190.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [52]:
frames = [start3811to3905, start3906to4000, start4001to4095, start4096to4190]
FinalData=pd.concat(frames)

In [53]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\LaLiga1718.csv')

## LaLiga 18-19

In [110]:
start4476to4555 = pd.read_csv("start4476to4555.csv",encoding='cp1252')
start4476to4555.head()
start4476to4555=start4476to4555.set_index('Match_ID')

In [111]:
print(start4191to4285.shape)
print(start4286to4380.shape)
print(start4381to4475.shape)
print(start4476to4555.shape)
print(start4556to4570.shape)

(95, 43)
(95, 43)
(95, 43)
(80, 43)
(15, 43)


In [112]:
frames = [start4191to4285, start4286to4380, start4381to4475, start4476to4555, start4556to4570]
FinalData=pd.concat(frames)

In [113]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\LaLiga1819.csv')

## LaLiga 19-20

In [161]:
start4856to4931 = pd.read_csv("start4856to4931.csv",encoding='cp1252')
start4856to4931.head()
start4856to4931=start4856to4931.set_index('Match_ID')

In [174]:
print(start4571to4665.shape)
print(start4666to4760.shape)
print(start4761to4855.shape)
print(start4856to4931.shape)
print(start4932to4950.shape)

(95, 43)
(95, 43)
(95, 43)
(76, 43)
(19, 43)


In [175]:
frames = [start4571to4665, start4666to4760, start4761to4855, start4856to4931, start4932to4950]
FinalData=pd.concat(frames)

In [176]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\LaLiga1920.csv')

## LaLiga 20-21

In [221]:
print(start4951to5045.shape)
print(start5046to5140.shape)
print(start5141to5235.shape)
print(start5236to5330.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [222]:
frames = [start4951to5045, start5046to5140, start5141to5235, start5236to5330]
FinalData=pd.concat(frames)

In [223]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\LaLiga2021.csv')

## Ligue1 16-17

In [49]:
print(start5331to5425.shape)
print(start5426to5520.shape)
print(start5521to5615.shape)
print(start5616to5710.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [50]:
frames = [start5331to5425, start5426to5520, start5521to5615, start5616to5710]
FinalData=pd.concat(frames)

In [108]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Ligue11617.csv')

## Ligue1 17-18

In [95]:
print(start5711to5805.shape)
print(start5806to5900.shape)
print(start5901to5995.shape)
print(start5996to6090.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [96]:
frames = [start5711to5805, start5806to5900, start5901to5995, start5996to6090]
FinalData=pd.concat(frames)

In [97]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Ligue11718.csv')

## Ligue1 18-19

In [154]:
print(start6091to6185.shape)
print(start6186to6280.shape)
print(start6281to6375.shape)
print(start6376to6470.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [155]:
frames = [start6091to6185, start6186to6280, start6281to6375, start6376to6470]
FinalData=pd.concat(frames)

In [156]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Ligue11819.csv')

## Ligue1 19-20

In [38]:
print(start6471to6565.shape)
print(start6566to6660.shape)
print(start6661to6749.shape)

(95, 43)
(95, 43)
(89, 43)


In [39]:
frames = [start6471to6565, start6566to6660, start6661to6749]
FinalData=pd.concat(frames)

In [40]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Ligue11920.csv')

## Ligue1 20-21

In [26]:
start6851to6877 = pd.read_csv("start6851to6877.csv",encoding='cp1252')
start6851to6877.head()
start6851to6877=start6851to6877.set_index('Match_ID')

In [50]:
print(start6851to6877.shape)
print(start6878to6946.shape)
print(start6947to7041.shape)
print(start7042to7136.shape)
print(start7137to7230.shape)

(27, 43)
(69, 43)
(95, 43)
(95, 43)
(94, 43)


In [51]:
frames = [start6851to6877, start6878to6946, start6947to7041, start7042to7136, start7137to7230]
FinalData=pd.concat(frames)

In [52]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\Ligue12021.csv')

## SerieA 16-17

In [96]:
print(start7231to7342.shape)
print(start7343to7437.shape)
print(start7438to7532.shape)
print(start7533to7610.shape)

(127, 43)
(95, 43)
(95, 43)
(78, 43)


In [97]:
frames = [start7231to7342, start7343to7437, start7438to7532, start7533to7610]
FinalData=pd.concat(frames)

In [98]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\SerieA1617.csv')

## SerieA 17-18

In [143]:
print(start7611to7705.shape)
print(start7706to7800.shape)
print(start7801to7895.shape)
print(start7896to7990.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [144]:
frames = [start7611to7705, start7706to7800, start7801to7895, start7896to7990]
FinalData=pd.concat(frames)

In [145]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\SerieA1718.csv')

## SerieA 18-19

In [197]:
print(start7991to8085.shape)
print(start8086to8180.shape)
print(start8181to8275.shape)
print(start8276to8370.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [198]:
frames = [start7991to8085, start8086to8180, start8181to8275, start8276to8370]
FinalData=pd.concat(frames)

In [199]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\SerieA1819.csv')

## SerieA 19-20

In [15]:
start8371to8465 = pd.read_csv("start8371to8465.csv",encoding='cp1252')
start8371to8465=start8371to8465.set_index('Match_ID')
start8466to8560 = pd.read_csv("start8466to8560.csv",encoding='cp1252')
start8466to8560=start8466to8560.set_index('Match_ID')
start8561to8655 = pd.read_csv("start8561to8655.csv",encoding='cp1252')
start8561to8655=start8561to8655.set_index('Match_ID')

In [16]:
print(start8371to8465.shape)
print(start8466to8560.shape)
print(start8561to8655.shape)
print(start8656to8750.shape)

(95, 43)
(95, 43)
(95, 43)
(95, 43)


In [17]:
frames = [start8371to8465, start8466to8560, start8561to8655, start8656to8750]
FinalData=pd.concat(frames)

In [18]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\SerieA1920.csv')

## SerieA 20-21

In [63]:
print(start8751to8845.shape)
print(start8846to8940.shape)
print(start8941to9035.shape)
print(start9036to9130.shape)

(94, 43)
(95, 43)
(95, 43)
(95, 43)


In [64]:
frames = [start8751to8845, start8846to8940, start8941to9035, start9036to9130]
FinalData=pd.concat(frames)

In [65]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\Final Project\WebScraper and Data\Capstone Data\SerieA2021.csv')

In [67]:
FinalData.index.value_counts()

9130    1
8881    1
8872    1
8873    1
8874    1
       ..
9006    1
9007    1
9008    1
9009    1
8751    1
Name: Match_ID, Length: 379, dtype: int64

In [ ]:
FinalData['']